要运行此操作，请按“*Runtime*”，然后按“*运行*” ** free ** tesla Tesla T4 Google COLAB实例！
<div class =“ Align-Center”>
<a href =“ https://unsloth.ai/”> <img src =“ https://github.com/unslothai/unslothai/unsloth/raw/raw/main/main/mains/unsloth%20new%20new%20logo.png.png”
<a href =“ https://discord.gg/unsloth”> <img src =“ https://github.com/unslothai/unslothai/unsloth/raw/main/main/main/images/images/discord button.png button.png
<a href =“ https://docs.unsloth.ai/”> <img src =“ https://github.com/unslothai/unslothai/unsloth/unsloth/main/main/main/mains/images/documentation%20green%20green%20breen%20button.png?png?raw=raw=true width width” <i>在<a href =“ https://github.com/unslothai/unsloth”> github </a> </i>⭐中
</div>

要在您自己的计算机上安装不塞，请按照我们的github页面上的安装说明[here](https://docs.unsloth.ai/get-started/installing-+-updating)上的安装说明。

您将学习如何做[data prep](#Data)，如何[train](#Train)，如何[run the model](#Inference)，＆xx_markDown_link_xx


### 消息

** New ** Unsploth现在支持培训OpenAi的新** GPT-oss **模型！您可以通过我们的** [Colab notebook](https://x.com/UnslothAI/status/1953896997867729075)免费启动Finetune GPT-oss **！

Unsploth现在支持文本对语音（TTS）模型。阅读我们的xx_markDown_link_xx。

阅读我们的** [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning) **并查看我们的新** [Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune) ** Quants优先量优于其他量化方法！

请访问我们的所有文档，以获取我们的所有XX_MarkDown_link_xx和[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)。


### 安装

In [ ]:
%%capture
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu124
!pip install unsloth
!pip install --upgrade transformers==4.53.2 "huggingface_hub>=0.34.0" "datasets>=3.4.1,<4.0.0"


### 不塞

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # 选择任何！我们在内部自动支持绳索扩展！
dtype = None # 没有自动检测。 Tesla T4，V100，Bfloat16的float16 for Ampere+
load_in_4bit = True # 使用4位量化来减少内存使用量。可以是错误的。

# 4位预量化模型我们支持4倍下载 + no ooms。
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # 新的Mistral V3 2X快速！
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3-15万亿代币2型快速！
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # PHI-3 2倍！
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2倍！
] # https://huggingface.co/unsloth的更多型号

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token =“ hf _...”，＃如果使用元模型，例如meta-llama/llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

现在，我们添加Lora适配器，因此我们只需要更新所有参数的1％至10％！

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # 选择任何数字> 0！建议8、16、32、64、128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # 支持任何人，但= 0已优化
    bias = "none",    # 支持任何人，但是=“无”
    # [NEW]“ Unsploth”使用的VRAM减少了30％，适合2倍批量的尺寸！
    use_gradient_checkpointing = "unsloth", # 在很长的背景下真实或“不整齐”
    random_state = 3407,
    use_rslora = False,  # 我们支持排名稳定的洛拉
    loftq_config = None, # 和Loftq
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name =“ data”> </a>
###数据准备
现在，我们使用XX_Markdown_link_xx的羊Alpaca数据集，该数据集是从GPT4生成的原始XX_Markdown_link_xx的52K版本。您可以用自己的数据准备替换此代码部分。

In [ ]:
from datasets import load_dataset

dataset = load_dataset("vicgalle/alpaca-gpt4", split="train")
print(dataset.column_names)

README.md:   0%|          | 0.00/3.38k [00:00<?, ?B/s]

(…)-00000-of-00001-6ef3991c06080e14.parquet:   0%|          | 0.00/48.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

['instruction', 'input', 'output', 'text']


一个问题是该数据集具有多个列。对于“ ollama”和“ llama.cpp”，要像自定义`chatgpt`聊天机器人一样函数，我们只能有2列 - `custemention'和`tought'''和`tougpt''列。

In [ ]:
print(dataset.column_names)

['instruction', 'input', 'output', 'text']


为了解决这个问题，我们将做以下操作：
*将所有列合并为1个说明提示。
*请记住，LLMS是文本预测指标，因此我们可以将指令自定义为我们喜欢的任何东西！
*使用“ to_sharegpt”函数进行此列合并过程！

例如，在我们的[Titanic CSV finetuning notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-olama.ipynb中，我们在1个提示中合并了多个列：

<img src =“ https://raw.githubusercontent.com/unslothai/unsloth/nightly/images/merge.png” height =“ 100”>

要将多列合并为1，请使用`merged_prompt`。
*将所有列包装在卷曲括号`{}`中。
*可选文本必须封闭在`[[]]`。例如，如果列“ PCLASS”为空，则合并函数将不会显示文本并将其SKP。这对于缺少值的数据集很有用。
*您可以选择每一列或几列！
*在“ output_column_name”中选择输出或目标 /预测列。对于羊驼数据集，这将是“输出”。

为了使Finetune处理多个转弯（例如在Chatgpt中），我们必须创建一个带有多个回合的“假”数据集 - 我们使用`consection_extension`从数据集中随机选择一些对话，然后将它们包装在一起中1个对话。

In [ ]:
from unsloth import to_sharegpt

dataset = to_sharegpt(
    dataset,
    merged_prompt="{instruction}[[\nYour input is:\n{input}]]",
    output_column_name="output",
    conversation_extension=3,  # 选择更多以处理更长的对话
)

Merging columns:   0%|          | 0/52002 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/52002 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/52002 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/52002 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/52002 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/52002 [00:00<?, ? examples/s]

最后，使用`sandardize_sharegpt'修复数据集！

In [ ]:
from unsloth import standardize_sharegpt

dataset = standardize_sharegpt(dataset)

Standardizing format:   0%|          | 0/52002 [00:00<?, ? examples/s]

### 可自定义的聊天模板

您还需要指定聊天模板。以前，您可以使用羊驼格式，如下所示。

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ＃＃ 操作说明：
{}

# ＃＃ 输入：
{}

# ＃＃ 回复：
{}"""

现在，您必须使用`{input}`作为指令，`{output}`作为响应。

我们还允许您使用可选的`{system}`字段。当您想使用自定义系统提示（也喜欢ChatGpt）时，这对Ollama很有用。

您也不能放置`{system}`字段，而只是放置纯文本。

```python
chat_template = """{SYSTEM}
USER: {INPUT}
ASSISTANT: {OUTPUT}"""
```

如果要使用Llama-3提示格式，请使用以下使用。如果您使用此此功能，则必须使用`指令'而不是``base''模型了！
```python
chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>

{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{OUTPUT}<|eot_id|>"""
```

对于ChatMl格式：
```python
chat_template = """<|im_start|>system
{SYSTEM}<|im_end|>
<|im_start|>user
{INPUT}<|im_end|>
<|im_start|>assistant
{OUTPUT}<|im_end|>"""
```

问题是羊驼格式有3个字段，而OpenAI样式聊天机器人只能使用2个字段（指令和响应）。这就是为什么我们使用“ to_sharegpt”函数将这些列合并为1的原因。

In [ ]:
chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

# ＃＃ 操作说明：
{INPUT}

# ＃＃ 回复：
{OUTPUT}"""

from unsloth import apply_chat_template

dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
    # default_system_message =“您是有用的助手”，<< [可选]
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

<a name =“ train”> </a>
###训练模型
现在，让我们使用huggingface trl的“ sfttrainer”！这里更多文档：[TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer)。我们执行60个步骤来加快速度，但是您可以将`num_train_epochs = 1`设置为完整运行，然后关闭`max_steps = none“。我们还支持TRL的“ Dpotrainer”！

In [ ]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # 可以使短序列更快地训练5倍。
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        # num_train_epochs = 1，＃对于更长的训练运行！
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # 将其用于Wandb等
    ),
)

Map (num_proc=2):   0%|          | 0/52002 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# @Title显示当前内存统计
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.613 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 52,002 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


In [ ]:
# @Title显示最终内存和时间统计
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

910.2379 seconds used for training.
15.17 minutes used for training.
Peak reserved memory = 8.361 GB.
Peak reserved memory for training = 2.748 GB.
Peak reserved memory % of max memory = 56.692 %.
Peak reserved memory for training % of max memory = 18.633 %.


<a name =“推理”> </a>
###推理
让我们运行模型！不塞的推论也使推断速度加快了2倍！您应该使用类似于您易经的提示，否则您可能会获得不好的结果！

In [ ]:
FastLanguageModel.for_inference(model) # 启用本地2倍更快的推理
messages = [                    # 在下面更改！
    {"role": "user", "content": "Continue the fibonacci sequence! Your input is 1, 1, 2, 3, 5, 8,"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


The next number in the Fibonacci sequence is 13.<|end_of_text|>


由于我们创建了一个实际的聊天机器人，因此您还可以通过手动添加用户和助手之间的交替对话来进行更长的对话！

In [ ]:
FastLanguageModel.for_inference(model) # 启用本地2倍更快的推理
messages = [                         # 在下面更改！
    {"role": "user",      "content": "Continue the fibonacci sequence! Your input is 1, 1, 2, 3, 5, 8"},
    {"role": "assistant", "content": "The fibonacci sequence continues as 13, 21, 34, 55 and 89."},
    {"role": "user",      "content": "What is France's tallest tower called?"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

France's tallest tower is called the Eiffel Tower.<|end_of_text|>


<a name =“ save”> </a>
###保存，加载固定模型
要将最终模型保存为Lora适配器，请使用HuggingFace的“ push_to_hub”进行在线保存或`save_pretaining'用于本地保存。

** [注意] **这仅保存洛拉适配器，而不是完整的模型。要节省16位或GGUF，请向下滚动！

In [ ]:
model.save_pretrained("lora_model")  # 本地节省
tokenizer.save_pretrained("lora_model")
# model.push_to_hub（“ your_name/lora_model”，token =“ ...”）＃在线保存
# tokenizer.push_to_hub（“ your_name/lora_model”，token =“ ...”）＃在线保存

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

现在，如果您想加载洛拉适配器，我们刚刚保存用于推理，请将`false``设置为true`：：true'：

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # 您用于培训的模型
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # 启用本地2倍更快的推理
pass

messages = [                    # 在下面更改！
    {"role": "user", "content": "Describe anything special about a sequence. Your input is 1, 1, 2, 3, 5, 8,"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

The sequence 1, 1, 2, 3, 5, 8 is a special sequence known as the Fibonacci sequence. The Fibonacci sequence is a series of numbers where each number is the sum of the two previous numbers, starting with 0 and 1. In this case, the sequence is 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, and so on. The Fibonacci sequence has many interesting properties and is widely studied in mathematics and computer science.<|end_of_text|>


您也可以使用拥抱脸的“ AutomodelForpeftCausAllm”。仅在未安装``'''''时就使用。由于不支持“ 4bit”模型下载，因此它可以慢慢放慢，并且不舒服的**推理快2倍**。

In [ ]:
if False:
    # 我高度建议 - 如果可能的话，请使用不塞
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # 您用于培训的模型
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

<a name =“ ollama”> </a>
### Ollama支持

[Unsloth](https://github.com/unslothai/unsloth)现在允许您自动finetune并创建XX_Markdown_link_xx，并导出到[Modelfile](https://github.com/ollama/ollama/blob/main/docs/modelfile.md)！这使得芬太尼变得更加容易，并提供了从``unsploth''到``ollama''的无缝工作流程！

让我们首先安装“ Ollama”！

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


接下来，我们将将模型保存到gguf / llama.cpp

我们克隆“ llama.cpp”，默认将其保存到Q8_0`。我们允许所有方法'q4_k_m`。使用`save_pretained_gguf`进行本地保存，然后`push_to_hub_gguf`将上传到HF。

一些支持的量化方法（我们的[Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)上的完整列表）：
*`Q8_0`-快速转换。大量资源使用，但通常可以接受。
*`q4_k_m`-推荐。使用q6_k作为注意到的一半。
*`q5_k_m`-推荐。使用q6_k进行一半的注意。wv和feed_forward.w2张量，else q5_k。

我们还支持以列表方式节省多个GGUF选项！如果您想要多种导出格式，这可以加快10分钟或更长时间的速度！

In [ ]:
# 保存到8位Q8_0
if True: model.save_pretrained_gguf("model", tokenizer,)
# 请记住要访问https://huggingface.co/settings/tokens for代币！
# 并将HF更改为您的用户名！
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# 保存至16位GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# 保存到Q4_K_M GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# 保存到多个GGUF选项 - 如果要多个，请更快！
if False:
    model.push_to_hub_gguf(
        "hf/model", # 将HF更改为您的用户名！
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # 在https://huggingface.co/settings/tokens上获取一个令牌
    )

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.
Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.49 out of 12.67 RAM for saving.


100%|██████████| 32/32 [01:41<00:00,  3.18s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be ./model/unsloth.Q8_0.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> Q8_0, shape = {4096, 128256}
INFO:hf-to-gguf:blk.0.attn_q.weight,         torch.float16 -

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: ./model/unsloth.Q8_0.gguf
Unsloth: Saved Ollama Modelfile to model/Modelfile


我们使用``subprocess''以非阻止方式开始``ollama''！在您自己的桌面中，您只需打开一个新的“终端”并键入``Ollama serve''，但是在Colab中，我们必须使用此黑客！

In [ ]:
import subprocess

subprocess.Popen(["ollama", "serve"])
import time

time.sleep(3)  # 等待Ollama加载几秒钟！

“ Ollama”需要一个``Modelfile''，它指定了模型的及时格式。让我们打印不塞的自动生成：

In [ ]:
print(tokenizer._ollama_modelfile)

FROM {__FILE_LOCATION__}

TEMPLATE """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.{{ if .Prompt }}

### Instruction:
{{ .Prompt }}{{ end }}

### Response:
{{ .Response }}<|end_of_text|>"""

PARAMETER stop "<|eot_id|>"
PARAMETER stop "<|start_header_id|>"
PARAMETER stop "<|end_header_id|>"
PARAMETER stop "<|end_of_text|>"
PARAMETER stop "<|reserved_special_token_"
PARAMETER temperature 1.5
PARAMETER min_p 0.1


现在，我们将使用我们自动生成的“ Modelfile”创建一个称为`unsloth_model`的ollama'模型！

In [ ]:
!ollama create unsloth_model -f ./model/Modelfile

transferring model data ⠋ transferring model data ⠹ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠧ transferring model data ⠇ transferring model data ⠏ transferring model data ⠋ transferring model data ⠋ transferring model data ⠙ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠦ transferring model data ⠧ transferring model data ⠏ transferring model data ⠏ transferring model data ⠋ transferring model data ⠙ transferring model data ⠹ transferring model data ⠼ transferring model data ⠼ transferring model data ⠦ transferring model data ⠧ transferring model data ⠧ transferring model data ⠏ transferring model data ⠏ transferring model data ⠋ transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠴ transferring model data ⠴ transferring model data ⠧ transferring model data ⠇ transferring

现在，我们可以通过“ Ollama”对此进行推断！

您还可以上传到``ollama''，然后尝试https://www.ollama.com/尝试`ollama`桌面应用程序

In [ ]:
!curl http://localhost:11434/api/chat -d '{ \
    "model": "unsloth_model", \
    "messages": [ \
        { "role": "user", "content": "Continue the Fibonacci sequence: 1, 1, 2, 3, 5, 8," } \
    ] \
    }'

{"model":"unsloth_model","created_at":"2024-10-01T06:47:04.241326628Z","message":{"role":"assistant","content":"The"},"done":false}
{"model":"unsloth_model","created_at":"2024-10-01T06:47:04.465575479Z","message":{"role":"assistant","content":" next"},"done":false}
{"model":"unsloth_model","created_at":"2024-10-01T06:47:04.760101468Z","message":{"role":"assistant","content":" number"},"done":false}
{"model":"unsloth_model","created_at":"2024-10-01T06:47:05.051240606Z","message":{"role":"assistant","content":" in"},"done":false}
{"model":"unsloth_model","created_at":"2024-10-01T06:47:05.376545126Z","message":{"role":"assistant","content":" the"},"done":false}
{"model":"unsloth_model","created_at":"2024-10-01T06:47:05.515751946Z","message":{"role":"assistant","content":" Fibonacci"},"done":false}
{"model":"unsloth_model","created_at":"2024-10-01T06:47:05.658721744Z","message":{"role":"assistant","content":" sequence"},"done":false}
{"model":"unsloth_model","created_at":"2024-10-01T06:47:

＃chatgpt互动模式

###⭐要像Chatgpt样式接口一样运行Finetuned Model，请先单击** | > _ | **按钮。
！

---
---
---

###⭐⭐然后键入`ollama run unsloth_model`

！

---
---
---
###⭐⭐，您有一个Chatgpt风格的助手！

###键入您喜欢的任何问题，然后按`Enter'。如果您想退出，请点击`ctrl + d`
！您可以安装JAN XX_MARKDOWN_LINK_XX并打开webui [here](https://github.com/janhq/jan)

我们完成了！如果您对不舒服有任何疑问，我们有一个XX_MarkDown_link_xx频道！如果您发现任何错误或想对最新的LLM内容进行更新，或者需要帮助，加入项目等，请随时加入我们的不和谐！

其他一些链接：
1。训练您自己的推理模型-Llama grpo笔记本XX_Markdown_link_xx-grpo.ipynb）
2。将芬太尼保存到奥尔马。 [here](https://github.com/open-webui/open-webui)-ollama.ipynb）
3。Llama3.2视觉燃烧 -  X射线照相用例。 [Discord](https://discord.gg/unsloth)-vision.ipynb）
6。请参见DPO，ORPO，继续预处理，对话finetuning等笔记本，以及我们的xx_markDown_link_xx上的更多内容！

<div class =“ Align-Center”>
  <a href =“ https://unsloth.ai”> <img src =“ https://github.com/unslothai/unslothai/unsloth/raw/raw/main/main/images/unsloth%20new%20new%20logo.png”
  <a href =“ https://discord.gg/unsloth”> <img src =“ https://github.com/unslothai/unsloth/unsloth/raw/main/main/main/images/images/discord.png，png
  <a href =“ https://docs.unsloth.ai/”> <img src =“ https://github.com/unslothai/unslothai/unsloth/unsloth/main/main/main/images/images/images/documentation%20green%20green%20breen%20button.png?png?png?raw=true width =”

  如果您需要帮助，请加入DISCORD +⭐️<i>在<a href =“ https://github.com/unslothai/unsloth上”
</div>
